In [1]:
import pandas as pd

# 1. Chargement du dataset

In [2]:
datas = pd.read_csv("../datas/prediction_disponibilite_et_demande.csv")
datas.head(5)

,annonce_id,gp_id,date_snapshot,date_depart,origin,destination,kilos_total,kilos_reserved_before,kilos_remaining,occupancy_rate,days_until_departure,dow_snapshot,is_weekend,month_snapshot,hist_bookings_7d,hist_bookings_30d,route_bookings_30d,kilos_booked_next_day
0,2,1,2025-11-03 07:43:00,2026-01-02 07:43:00,Paris,Casa,24,0.0,24.0,0.0,60,0,False,11,0.0,0.0,9,0.0
1,2,1,2025-11-04 07:43:00,2026-01-02 07:43:00,Paris,Casa,24,0.0,24.0,0.0,59,1,False,11,0.0,0.0,9,0.0
2,2,1,2025-11-05 07:43:00,2026-01-02 07:43:00,Paris,Casa,24,0.0,24.0,0.0,58,2,False,11,0.0,0.0,9,0.0
3,2,1,2025-11-06 07:43:00,2026-01-02 07:43:00,Paris,Casa,24,0.0,24.0,0.0,57,3,False,11,0.0,0.0,9,0.0
4,2,1,2025-11-07 07:43:00,2026-01-02 07:43:00,Paris,Casa,24,0.0,24.0,0.0,56,4,False,11,0.0,0.0,9,0.0


# 2. Analyse exploratoire des données

In [3]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   annonce_id             298 non-null    int64  
 1   gp_id                  298 non-null    int64  
 2   date_snapshot          298 non-null    object 
 3   date_depart            298 non-null    object 
 4   origin                 298 non-null    object 
 5   destination            298 non-null    object 
 6   kilos_total            298 non-null    int64  
 7   kilos_reserved_before  298 non-null    float64
 8   kilos_remaining        298 non-null    float64
 9   occupancy_rate         298 non-null    float64
 10  days_until_departure   298 non-null    int64  
 11  dow_snapshot           298 non-null    int64  
 12  is_weekend             298 non-null    bool   
 13  month_snapshot         298 non-null    int64  
 14  hist_bookings_7d       298 non-null    float64
 15  hist_b

In [4]:
datas.columns.tolist()

['annonce_id',
 'gp_id',
 'date_snapshot',
 'date_depart',
 'origin',
 'destination',
 'kilos_total',
 'kilos_reserved_before',
 'kilos_remaining',
 'occupancy_rate',
 'days_until_departure',
 'dow_snapshot',
 'is_weekend',
 'month_snapshot',
 'hist_bookings_7d',
 'hist_bookings_30d',
 'route_bookings_30d',
 'kilos_booked_next_day']

In [5]:
datas.duplicated().sum()

np.int64(0)

In [6]:
datas.isna().sum()

annonce_id               0
gp_id                    0
date_snapshot            0
date_depart              0
origin                   0
destination              0
kilos_total              0
kilos_reserved_before    0
kilos_remaining          0
occupancy_rate           0
days_until_departure     0
dow_snapshot             0
is_weekend               0
month_snapshot           0
hist_bookings_7d         0
hist_bookings_30d        0
route_bookings_30d       0
kilos_booked_next_day    0
dtype: int64

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
def outliers(dframe):
    outliers = dframe.copy()
    for i in dframe.select_dtypes(include="number").columns.tolist():
        q1 = dframe[i].quantile(0.25)
        q3 = dframe[i].quantile(0.75)
        iqr = q3 - q1
        b_inf = q1 - 1.5*iqr
        b_sup = q3 + 1.5*iqr
        outliers = dframe[(dframe[i] < b_inf) | (dframe[i] > b_sup)]
    return outliers
outliers = outliers(datas)
outliers

,annonce_id,gp_id,date_snapshot,date_depart,origin,destination,kilos_total,kilos_reserved_before,kilos_remaining,occupancy_rate,days_until_departure,dow_snapshot,is_weekend,month_snapshot,hist_bookings_7d,hist_bookings_30d,route_bookings_30d,kilos_booked_next_day
57,2,1,2025-12-30 07:43:00,2026-01-02 07:43:00,Paris,Casa,24,0.0,24.0,0.0,3,1,False,12,0.0,0.0,9,24.0
297,5,1,2026-01-11 07:00:00,2026-01-14 07:00:00,Dakar,Paris,16,0.0,16.0,0.0,3,6,True,1,0.0,0.0,3,16.0


# 3. Entrainement

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score
y = datas["kilos_booked_next_day"]
X = datas.drop(
    columns=[
        "kilos_booked_next_day",
        "annonce_id",
        "gp_id",
        "date_snapshot",
        "date_depart"
    ]
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import numpy as np
import os
import joblib

cat_features = ["origin", "destination", "is_weekend"]
num_features = [
    "kilos_total",
    "kilos_reserved_before",
    "kilos_remaining",
    "occupancy_rate",
    "days_until_departure",
    "dow_snapshot",
    "month_snapshot",
    "hist_bookings_7d",
    "hist_bookings_30d",
    "route_bookings_30d"
]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

# Définir plusieurs modèles à tester
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_estimators=200, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=200, random_state=42)
}

# Dictionnaire pour stocker les scores
model_scores = {}

for name, model in models.items():
    # Pipeline complet
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # Entraînement
    pipeline.fit(X_train, y_train)
    
    # Prédiction
    y_pred = pipeline.predict(X_test)
    
    # Évaluation
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    #rmse = mean_squared_error(y_test, y_pred, squared=False)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    r2 = r2_score(y_test, y_pred)
    
    model_scores[name] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    }
    
    print(f"=== {name} ===")
    print(f"MAE: {mae:.3f}, RMSE: {rmse:.3f}, R2: {r2:.3f}\n")

# Sélection du meilleur modèle selon R2 (ou MAE selon besoin)
best_model_name = max(model_scores, key=lambda x: model_scores[x]['R2'])
print(f"Meilleur modèle selon R2: {best_model_name}")

# Enregistrement du pipeline complet du meilleur modèle
best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', models[best_model_name])
])

best_pipeline.fit(X, y)  # on réentraîne sur tout le dataset

# Dossier où enregistrer le pipeline
model_dir = "../models/"
os.makedirs(model_dir, exist_ok=True)  

# Enregistrement du pipeline
joblib.dump(best_pipeline, os.path.join(model_dir, f"{best_model_name}_pipeline.pkl"))
print(f"Pipeline du modèle {best_model_name} enregistrée !")


=== LinearRegression ===
MAE: 0.544, RMSE: 3.106, R2: -0.022

=== RandomForest ===
MAE: 0.401, RMSE: 3.068, R2: 0.003

=== GradientBoosting ===
MAE: 0.400, RMSE: 3.098, R2: -0.017

Meilleur modèle selon R2: RandomForest
Pipeline du modèle RandomForest enregistrée !
